In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install langchainhub
!pip install openai
!pip install python-dotenv
!pip install langchain
!pip install langchain_community
!pip install -qU langchain-openai
!pip install faiss-cpu
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 699.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv('/content/drive/Othercomputers/내 노트북/code/MarketingAI-Bot/.env')

True

In [10]:
import pandas as pd
# 원본 CSV 파일 경로
original_file_path = "/content/drive/Othercomputers/내 노트북/code/MarketingAI-Bot/output2.csv"
filtered_file_path = "./filtered_output.csv"

# pandas를 사용하여 CSV 파일 로드
df = pd.read_csv(original_file_path)

# 해쉬태그 컬럼에서 '#이벤트' 또는 '#EVENT'를 포함하는 행 필터링
df_filtered = df[~df['해쉬태그'].str.contains('#이벤트|#EVENT', case=False, na=False)]

# 필터링된 데이터를 임시 CSV 파일로 저장
df_filtered.to_csv(filtered_file_path, index=False)
df_filtered.head(5)

,Index,생성시간,게시글내용,해쉬태그,img_path
0,1,2024.07.05 08:52:57,𝙉𝙚𝙬𝙡𝙮 𝙡𝙖𝙪𝙣𝙘𝙝!어글리 러블리 마스크팩 트래블러스 키트 출시💃어글리 러블리 ...,"['#어글리러블리', '#uglylovely']",./images\0.jpg
1,2,2024.07.02 08:18:08,두 가지 새로운 소식이 도착했어요!🙋‍♀️첫 번째는 이제 어글리 러블리 뷰티 제품들...,"['#어글리러블리', '#uglylovely']",./images\1.jpg
2,3,2024.06.28 02:28:00,코 앞으로 다가온 여름 장마☔이번 주말에는 무더운 날씨를 한 풀 식혀줄 비 소식이 ...,"['#어글리러블리', '#uglylovely']",./images\2.jpg
3,4,2024.06.21 08:12:49,"일년 중 가장 낮이 긴 날, 여름의 절기 하지☀️어글리 러블리 농작물들도 햇빛을 받...","['#어글리러블리', '#uglylovely']",./images\3.jpg
4,5,2024.06.18 08:09:00,"먹지 못하는 농작물의 껍질, 씨앗도 버리지 않고피부에 이로운 원료로 새-활용합니다....","['#어글리러블리', '#uglylovely']",./images\4.jpg


In [9]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import os

loader = CSVLoader(file_path=filtered_file_path)
data = loader.load()
print(len(data))

GPT_API_KEY = os.environ.get('GPT_API_KEY')
ORG_ID = os.environ.get('ORG_ID')

63


In [8]:
data[0]

Document(metadata={'source': './filtered_output.csv', 'row': 0}, page_content="Index: 1\n생성시간: 2024.07.05 08:52:57\n게시글내용: 𝙉𝙚𝙬𝙡𝙮 𝙡𝙖𝙪𝙣𝙘𝙝!어글리 러블리 마스크팩 트래블러스 키트 출시💃어글리 러블리 마스크 팩 3종 모두를파우치에 쏙, 여행 시 휴대가 간편한30ml 미니어처 사이즈로 만나보세요.사랑스러운 패키지 디자인을 적용해소중한 사람들에게, 늘 고마운 나에게선물해보시는 건 어떨까요?*마스크팩 3종 트래블러스 키트는 카카오톡 선물하기에서 만나보실 수 있습니다.\n해쉬태그: ['#어글리러블리', '#uglylovely']\nimg_path: ./images\\0.jpg")

In [11]:
from langchain_openai import OpenAIEmbeddings

# OpenAI의 "text-embedding-ada-002" 모델을 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002",openai_api_key=GPT_API_KEY)

In [12]:
from langchain_community.vectorstores import FAISS

# 저장
vectorstore = FAISS.from_documents(data, embeddings)

# 저장된 DB의 리트리버를 가져와서 검색에 사용한다.
# 서치 타입 mmr은 유사한 정보를 가진 문서는 제외하는 알고리즘을 사용하는것이다.
# 검색인자로 k는 반환할 문서의 수를 지정한다.
# retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 5})

# 유사도를 이용한 retiever 생성
retriever = vectorstore.as_retriever(
    # 검색 유형을 "유사도 점수 임계값"으로 설정합니다.
    search_type="similarity_score_threshold",
    # 검색 인자로 점수 임계값을 0.5로 지정합니다.
    search_kwargs={
        "score_threshold": 0.5,
        "k": 7},
)

In [13]:
# FAISS 모듈에서 인덱스를 가져옴
index = vectorstore.index

# 인덱스에 저장된 벡터의 개수를 출력
vector_count = index.ntotal
print(f"백터스토어에 저장된 데이터 개수: {vector_count}")

백터스토어에 저장된 데이터 개수: 63


In [14]:
# 검색된 컨택스트를 하나로 합쳐준다.
def format_docs(docs):
  result= []
  for doc in docs:
    page_content = doc.page_content
    result.append(page_content)
    # print(page_content)
  return "\n\n".join(result)

# 문서 검색 및 포맷팅
def search_and_format(retriever,question):
    docs = retriever.invoke(question)
    context = format_docs(docs)
    return {"context": context, "question": question}

In [15]:
docs = retriever.invoke("유자")

In [16]:
import json
import re

def convert_to_json(string):
    # 문자열에서 각 요소를 추출하는 정규식 패턴
    pattern = r"Index: (\d+)\n생성시간: ([\d.:\s]+)\n게시글내용: ([^\n]+)\n해쉬태그: \[([^\]]+)\]\nimg_path: ([^\n]+)"
    match = re.search(pattern, string)

    if not match:
        raise ValueError("String format is incorrect")

    index = int(match.group(1))
    생성시간 = match.group(2)
    게시글내용 = match.group(3)
    해쉬태그 = match.group(4).replace("'", "").split(", ")
    img_path = match.group(5).replace("\\", "/")

    # JSON 객체 생성
    json_data = {
        "Index": index,
        "생성시간": 생성시간,
        "게시글내용": 게시글내용,
        "해쉬태그": 해쉬태그,
        "img_path": img_path
    }
    return json_data



In [17]:
context_and_question = search_and_format(retriever,"유자처럼 상큼한것이 없을까?")
print("-"*100)
context_and_question

----------------------------------------------------------------------------------------------------


{'context': "Index: 7\n생성시간: 2024.06.11 07:47:31\n게시글내용: 유자청을 닮은 찐~득한 제형이피부를 맑고 환하게 밀착케어 해줍니다.새콤 달콤한 유자 향으로 즐기는 향기 테라피는 덤!🍋\n해쉬태그: ['#어글리러블리', '#uglylovely']\nimg_path: ./images\\6.jpg\n\nIndex: 62\n생성시간: 2023.12.04 07:52:28\n게시글내용: 유기농, 무농약으로 키워 삐뚤빼뚤 개성있는 생김새로 자란 못난이 무화과에 주목!는 상품 가치가 떨어져 버려지는 못난이 무화과에서 추출한 업사이클링 원료를 사용합니다.\n해쉬태그: ['#어글리러블리피그마스크', '#어글리러블리', '#uglylovely']\nimg_path: ./images\\61.jpg\n\nIndex: 9\n생성시간: 2024.06.07 05:31:50\n게시글내용: 그려 나 고흥 유자여!전남 고흥 못난이 유자를 업사이클링한 원료로 만든어글리 러블리 유자 마스크가 당신의 피부를 맑고 환하게\n해쉬태그: ['#어글리러블리', '#uglylovely']\nimg_path: ./images\\8.jpg\n\nIndex: 65\n생성시간: 2023.11.28 09:46:57\n게시글내용: 화학비료와 농약을 사용하지 않아 자연적으로 생긴 삐뚤빼뚤 영광의 상처를 가진 못난이 유자에 주목!는 상품 가치가 떨어져 버려지는 못난이 유자에서 추출한 업사이클링 원료를 사용합니다.네이버 스토어 단독 𝟮𝟵% 𝗢𝗙𝗙👆🏻상단 프로필 링크 클릭!\n해쉬태그: ['#어글리러블리유자마스크', '#어글리러블리', '#uglylovely']\nimg_path: ./images\\64.jpg\n\nIndex: 21\n생성시간: 2024.04.30 08:21:40\n게시글내용: 유자청을 닮아 찐~득한 텍스처를 가진어글리 러블리 유자 마스크 팩지친 피부에 깊은 보습감과 은은한 광채감으로생기 있는 피부를 경험해보세요.\n해쉬태그: ['#어글리러블리', '#uglylovely

In [29]:
# 랭스미스의 허브에 들어가면 미리 정의된 다양한 프롬프트들이 있다.
# hum를 통해 해당 프롬프트 탬플릿을 사용할 수 있다.

# 랭스미스의 허브에 들어가면 미리 정의된 다양한 프롬프트들이 있다.
# hum를 통해 해당 프롬프트 탬플릿을 사용할 수 있다.
from langchain import hub

# 프롬프트 템플릿 불러오기
prompt = hub.pull("rlm/rag-prompt")

# 불러온 프롬프트 출력 (원래의 템플릿을 확인하기 위해)
print("Original Prompt:", prompt.messages[0].prompt.template)

# 추가하고 싶은 영어 프롬프트 내용
additional_content = "\nPlease ensure to provide detailed references and explanations in a concise manner."
prompt.messages[0].prompt.template += additional_content

# 수정된 프롬프트 출력
print("Modified Prompt:", prompt)

# 만약 프롬프트에 변수를 삽입하고 싶다면 아래와 같이 사용합니다.
# final_prompt = modified_prompt.format(variable_name="value")

# print("Final Prompt:", final_prompt)

Original Prompt: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
Modified Prompt: input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:\nPlease ensure to provide detailed references and explanations in a concise manner.")

In [30]:
prompt.format(context = "sdf", question ="adf")

"Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: adf \nContext: sdf \nAnswer:\nPlease ensure to provide detailed references and explanations in a concise manner."

In [32]:
rag_chain.invoke("어글리러블리유자마스크를 여름에 홍보하기위한 홍보글을 작성해줘")

'여름철 더위와 습기를 이겨내고 싶으신가요? 🌞🍋 "어글리 러블리 유자 마스크"로 여름철 피부 관리를 시작해보세요! 고흥산 못난이 유자 추출물이 함유된 이 비건 마스크팩은 피부에 생기와 보습을 선사하며, 향긋한 유자의 향기로 기분까지 상쾌하게 만들어 줍니다. 지금 바로 네이버 스토어에서 29% 할인 혜택을 누리세요! 🌟 #어글리러블리유자마스크 #여름피부케어 #비건스킨케어'

In [31]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableSequence

llm = ChatOpenAI(model="gpt-4-turbo",openai_api_key=GPT_API_KEY)

rag_chain = (
    {"context":retriever | format_docs, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [33]:
query = "어글리러블리유자마스크를 여름에 홍보하기위한 홍보글을 작성해줘"#"유자"
docs = retriever.invoke(query)

llm = ChatOpenAI(model="gpt-4-turbo",openai_api_key=GPT_API_KEY)

result = prompt.format(context=format_docs(docs), question= query)
print(result)
result = llm( result)
print(result)
arr= []
for doc in docs:
  arr.append(convert_to_json(doc.page_content))
response_data = {"section1":arr,"section2":result.content}
print(response_data)

Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: 어글리러블리유자마스크를 여름에 홍보하기위한 홍보글을 작성해줘 
Context: Index: 3
생성시간: 2024.06.28 02:28:00
게시글내용: 코 앞으로 다가온 여름 장마☔이번 주말에는 무더운 날씨를 한 풀 식혀줄 비 소식이 있네요.외출하실 땐 우산 잊지 마시고,집에 계실 땐 어글리 러블리 마스크 팩으로기분 좋은 향기와 함께 스페셜 케어를 즐겨보시는 건 어떨까요?
해쉬태그: ['#어글리러블리', '#uglylovely']
img_path: ./images\2.jpg

Index: 19
생성시간: 2024.05.07 08:22:24
게시글내용: 기어, 나 제주 당근 마씸제주 구좌읍 못난이 당근을 업사이클링한 원료로 만든어글리 러블리 캐롯 마스크가 당신이 잠든 사이 보습열일
해쉬태그: ['#어글리러블리', '#uglylovely']
img_path: ./images\18.jpg

Index: 64
생성시간: 2023.11.29 08:04:25
게시글내용: 어제보다 밝은 오늘을 꿈꾸며✨유자스팀 생기톤업 비건 온열 보습팩𝑉𝑒𝑔𝑎𝑛 𝑤𝑎𝑠ℎ-𝑜𝑓𝑓 𝑚𝑎𝑠𝑘 𝑝𝑎𝑐𝑘 𝑐𝑜𝑛𝑡𝑎𝑖𝑛𝑖𝑛𝑔 𝑒𝑥𝑡𝑟𝑎𝑐𝑡 𝑜𝑓 𝑢𝑔𝑙𝑦 𝑦𝑢𝑗𝑎, 𝑤𝑖𝑡ℎ 𝑡ℎ𝑒 𝑡ℎ𝑖𝑐𝑘 𝑡𝑒𝑥𝑡𝑢𝑟𝑒 𝑓𝑒𝑒𝑙𝑖𝑛𝑔 𝑡ℎ𝑎𝑡 𝑖𝑠 𝑎𝑙𝑚𝑜𝑠𝑡 𝑙𝑖𝑘𝑒 𝑦𝑢𝑗𝑎 𝑠𝑦𝑟𝑢𝑝 𝑡𝑜 𝑐𝑟𝑒𝑎𝑡𝑒 𝑏𝑟𝑖𝑔ℎ𝑡 𝑎𝑛𝑑 ℎ𝑒𝑎𝑙𝑡ℎ𝑦 𝑔𝑙𝑜𝑤𝑖𝑛𝑔 𝑠𝑘𝑖𝑛.어글리 러블리 유자 마스크는고흥산 못난이 유자로 만든 비건 워시오프 마스크팩으로,피부 컨디션 회복에 도움을

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


content='여름철 무더위 속에서도 피부는 촉촉하고 생기 있게! 어글리 러블리 유자 마스크로 당신의 피부를 한층 더 밝고 건강하게 가꿔보세요. 고흥산 못난이 유자의 풍부한 비타민과 보습 성분이 피부에 깊은 영양을 전달하며, 지친 피부에 활력을 불어넣어 줍니다. #어글리러블리유자마스크 #여름스킨케어' response_metadata={'token_usage': {'completion_tokens': 178, 'prompt_tokens': 2086, 'total_tokens': 2264}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_595e3bc347', 'finish_reason': 'stop', 'logprobs': None} id='run-bb0256ef-7725-42e9-a4b4-2e66dd1b5dc1-0' usage_metadata={'input_tokens': 2086, 'output_tokens': 178, 'total_tokens': 2264}
{'section1': [{'Index': 3, '생성시간': '2024.06.28 02:28:00', '게시글내용': '코 앞으로 다가온 여름 장마☔이번 주말에는 무더운 날씨를 한 풀 식혀줄 비 소식이 있네요.외출하실 땐 우산 잊지 마시고,집에 계실 땐 어글리 러블리 마스크 팩으로기분 좋은 향기와 함께 스페셜 케어를 즐겨보시는 건 어떨까요?', '해쉬태그': ['#어글리러블리', '#uglylovely'], 'img_path': './images/2.jpg'}, {'Index': 19, '생성시간': '2024.05.07 08:22:24', '게시글내용': '기어, 나 제주 당근 마씸제주 구좌읍 못난이 당근을 업사이클링한 원료로 만든어글리 러블리 캐롯 마스크가 당신이 잠든 사이 보습열일', '해쉬태그': ['#어글리러블리', '#uglylovely'], 'img_path': './images/18.jpg'}, {'Index': 6

In [ ]:

response_data = {"section1":arr,"section2":result.content}
response_data

{'section1': [{'Index': 8,
   '생성시간': '2024.06.10 06:30:42',
   '게시글내용': '유자C의 브라이트닝 케어🤩세린, 아르기닌을 공융혼합한독자 개발 원료*를 함유한 유자 마스크 팩으로 효과적인 각질 케어와 함께 매끈해진 피부결을 경험해보세요.(*특허 제10-2269121호)',
   '해쉬태그': ['#어글리러블리', '#uglylovely'],
   'img_path': './images/7.jpg'},
  {'Index': 7,
   '생성시간': '2024.06.11 07:47:31',
   '게시글내용': '유자청을 닮은 찐~득한 제형이피부를 맑고 환하게 밀착케어 해줍니다.새콤 달콤한 유자 향으로 즐기는 향기 테라피는 덤!🍋',
   '해쉬태그': ['#어글리러블리', '#uglylovely'],
   'img_path': './images/6.jpg'},
  {'Index': 63,
   '생성시간': '2023.11.30 08:05:33',
   '게시글내용': '유자스팀 생기톤업 비건 온열 보습팩💡뚜껑을 열자마자 상큼 달달 유자향이 물씬,유자청을 떠올리게 하는 찐~득한 텍스처에피부에 전해지는 은은한 온열감까지!어글리 러블리와 함께 하는 모든 순간의 오감을 자극합니다.네이버 스토어 단독 𝟮𝟵% 𝗢𝗙𝗙👆🏻상단 프로필 링크 클릭!',
   '해쉬태그': ['#어글리러블리', '#uglylovely'],
   'img_path': './images/62.jpg'},
  {'Index': 9,
   '생성시간': '2024.06.07 05:31:50',
   '게시글내용': '그려 나 고흥 유자여!전남 고흥 못난이 유자를 업사이클링한 원료로 만든어글리 러블리 유자 마스크가 당신의 피부를 맑고 환하게',
   '해쉬태그': ['#어글리러블리', '#uglylovely'],
   'img_path': './images/8.jpg'},
  {'Index': 64,
   '생성시간': '2023.11.29 08:04: